In [ ]:
#Run any one cell to select a certain url

In [ ]:

import getpass
import os

password = getpass.getpass()
command = "sudo -S apt-get install chromium-chromedriver"
os.system('echo %s | %s' % (password, command))
command = "sudo -S apt-get update"
os.system('echo %s | %s' % (password, command))
command = "sudo -S apt-get install chromium-chromedriver"
os.system('echo %s | %s' % (password, command))

# If facing issues with the default firefox driver for webscraping using selenium , run the below code
# !sudo apt-get install chromium-chromedriver
# !sudo apt-get update
# !sudo apt-get install chromium-chromedriver
!pip install selenium
# Here the chrome driver is being used , hence replace firefox with Chrome driver as given below
!pip install webdriver-manager

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome(ChromeDriverManager().install(), options=options)

In [ ]:
#Alphabetical parse url
#url = "https://www.espncricinfo.com/ci/content/player/country.html?country=6;alpha=A"

In [3]:
from bs4 import BeautifulSoup as bf
import requests
import pandas as pd
import time
from selenium import webdriver

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# driver = webdriver.Firefox()
# driver = webdriver.Chrome(options=options)
driver = wd

In [133]:
u = "https://www.espncricinfo.com/player/ahmad-zazai-1151189"
re = requests.get(u)
#time.sleep(2)
so = bf(re.text,"html.parser")
l , i = evaluate(so)

In [16]:
def evaluate(html):
  select_divs = html.find_all('div',class_="card overflow-hidden mb-3")
  exist = 0
  location = 0
  for i in range(len(select_divs)):
    check = ""
    try:
      check = select_divs[i].find('p',class_="benton-bold pl-3 pt-4 pb-3 m-0 player-card-header").text
    except:
      k = 1
    if(check == "Career Averages"):
      exist = 1
      location = i
      break
  try:
    position = select_divs[location]
  except:
    position = ""
    exist = 0
  return exist,position

In [17]:
def get_player_image():
    # For fixing installation errors on linux: `sudo apt install firefox-geckodriver`
    global driver
    images = driver.find_elements_by_tag_name('img')
    images = [image for image in images if "player-card__face" in image.get_attribute("class")]
    if len(images) < 1:
        return ["image"], [""]
    print(images[0].get_attribute("src"))
    return ["image"], [images[0].get_attribute("src")]

In [18]:
# Get trophy info
def get_major_trophies_information(major_trophies_soup):
    major_trophies_results = {}
    all_h5_tags = major_trophies_soup.find_all('h5', class_="border-bottom-gray-300 m-0 pl-3 pb-2 table-header")
    if all_h5_tags is None:
        return None
    try:
        required_h5_tag = all_h5_tags[0]
    except:
        print('-'*50)
        print(all_h5_tags)
        print('-'*50)
    for h5_tag in all_h5_tags:
        if str(h5_tag.text).strip() == "In Major Trophies":
            required_h5_tag = h5_tag
            break
    if str(required_h5_tag.text).strip() != "In Major Trophies":
        return ["Major trophies"], [[major_trophies_results]]
    major_trophies_table_head = required_h5_tag.findNext('thead')
    major_trophies_table_headings = major_trophies_table_head.find_all('th')
    major_trophies_table_headings_count = 0
    major_trophies_headings_list = []
    for table_heading in major_trophies_table_headings:
        major_trophies_headings_list.append(table_heading.text)
    major_trophies_table_headings_count = len(major_trophies_headings_list)
    # print('Major Trophies table Head count', major_trophies_table_headings_count)
    current_title = "HOME"
    major_trophies_table_body = major_trophies_table_head.findNext('tbody')
    major_trophies_table_data = major_trophies_table_body.find_all('td')
    t = 0
    for table_data in major_trophies_table_data:
        attribute_number = t % major_trophies_table_headings_count
        if(attribute_number != 0):
            major_trophies_results[current_title][major_trophies_headings_list[attribute_number]] = table_data.text
        else:
            current_title = table_data.text
            major_trophies_results[current_title] = {}
        t += 1
    print(major_trophies_results)
    return ["Major trophies"], [[major_trophies_results]]

In [19]:
# Statistical analysis - Home, Away stats and function call to trophy info
def get_statistical_analysis_and_trophy_info_and_image(req):
    analysis_labels = []
    analysis_results = []
    analysis_url = req.url + "/bowling-batting-stats"
    #analysis_url = "https://www.espncricinfo.com/player/virat-kohli-253802/bowling-batting-stats"
    
    # selecting dropdown option
    global driver
    driver.get(analysis_url)
    # loading page
    time.sleep(2)
    dropdowns = driver.find_elements_by_class_name('dropdown-container')
    for dropdown in dropdowns:
        is_format_dropdown = str(dropdown.find_elements_by_tag_name('button')[0].text).strip() in ["Test", "ODI", "T20I", "Test+ODI+T20I"]
        is_role_dropdown = str(dropdown.find_elements_by_tag_name('button')[0].text).strip() in ["Batting", "Bowling", "Fielding", "Allround"]
        if (not is_format_dropdown) and (not is_role_dropdown):
            continue
        required_span_text = "Test+ODI+T20I"
        if is_role_dropdown:
            required_span_text = "Allround"
        dropdown.find_elements_by_tag_name('button')[0].click()
        all_list_items = dropdown.find_elements_by_tag_name('li')
        for list_item in all_list_items:
            span_text = list_item.find_elements_by_tag_name('span')[0].text
            if span_text == required_span_text:
                list_item.click()
                break
    # waiting for changes to load
    time.sleep(3)
    analysis_page = driver.page_source
    analysis_soup = bf(analysis_page, "html.parser")
    
    all_h5_tags = analysis_soup.find_all('h5', class_="border-bottom-gray-300 m-0 pl-3 pb-2 table-header")
    print(all_h5_tags)
    if len(all_h5_tags) is not 0:
      try:
          required_h5_tag = all_h5_tags[0]
      except:
          print('-'*50)
          print(all_h5_tags)
          print('-'*50)
      for h5_tag in all_h5_tags:
          if str(h5_tag.text).strip() == "Home vs Away":
              required_h5_tag = h5_tag
              break
      if str(required_h5_tag.text).strip() != "Home vs Away":
          return get_major_trophies_information(analysis_soup)
      analysis_table_head = required_h5_tag.findNext('thead')
      analysis_table_headings = analysis_table_head.find_all('th')
      analysis_table_headings_count = 0
      analysis_table_headings_list = []
      for table_heading in analysis_table_headings:
          analysis_table_headings_list.append(table_heading.text)
      analysis_table_headings_count = len(analysis_table_headings_list)
      # print('Analysis table Head count', analysis_table_headings_count)
      current_title = "HOME"
      analysis_table_body = analysis_table_head.findNext('tbody')
      analysis_table_data = analysis_table_body.find_all('td')
      t = 0
      for table_data in analysis_table_data:
          attribute_number = t % analysis_table_headings_count
          if(attribute_number != 0):
              analysis_results.append(table_data.text)
              analysis_labels.append(current_title + "_" + analysis_table_headings_list[attribute_number])
          else:
              current_title = table_data.text.upper()
          t += 1
      trophy_label, trophy_result = get_major_trophies_information(analysis_soup)
      image_label, image_result = get_player_image()
    else:
      return [] , []
    # for i in range(len(analysis_labels)):
    #     print(f'{analysis_labels[i]} : {analysis_results[i]}')

    return analysis_labels + trophy_label + image_label, analysis_results + trophy_result + image_result

In [20]:
def find_relations(Relations):
    relations_container = Relations.find_next('div')
    individual_relations = relations_container.find_all('span')
    total_relations = list()
    for i in range(len(individual_relations)):
        name = individual_relations[i].find('h5').text
        bond = individual_relations[i].find('p').text
        total_relations.append([name,bond])
    return total_relations

In [21]:
#Get relevant information related to a player using each player's url
def load(req,final_list_players,ind,u,debut_url):
    lab = list()
    result = list()
    soup = bf(req.text,"html.parser")
    all_player_urls = [req.url]
    print(soup.title)
    search_tag_container = soup.find_all('h5',class_='player-card-description gray-900')
    count = 0
    for job_elem in search_tag_container:
        result.append(job_elem.text)
        count+=1
    search_tag_container = soup.find_all('p',class_="text-uppercase gray-700 mb-0 pb-0-5 player-card-heading")
    player_gender = "Male"
    for job_elem in search_tag_container:
        lab.append(job_elem.text)
    player_role = "xyz"
    for i in range(len(lab)):
        if "role" in lab[i].lower():
            player_role = result[i].lower()
            break
    ##################################
    valid, bat_bowl_soup = evaluate(soup)
    if valid != 1:
        return
    search_tag_container = bat_bowl_soup.findAll('thead')
    match_averages = []
    runs_scored_as_batsman = 0
    balls_bowled_as_bowler = 0
    for m in range(len(search_tag_container)):
        actual_heading = search_tag_container[m].findPrevious('h5').text
        current_string = "_"
        if "Bowling" in actual_heading:
            current_string = "Bowling_"
        elif "Batting" in actual_heading:
            current_string = "Batting_"
        else:
            continue
        match_records = {}
        tag_content = search_tag_container[m].find_all('th')
        head = 0
        table_labels = list()
        table_labels_1 =list()
        for job_elem in tag_content:
            table_labels.append(job_elem.text)
            head += 1
        print('Head count',head)

        tag_content = search_tag_container[m].findNext('tbody').find_all('td')
        track_head_label = 0
        current_format = ""
        for job_elem in tag_content:
            attribute_number = track_head_label % head
            if(attribute_number != 0):
                result.append(job_elem.text)
                label_text = table_labels[attribute_number]
                lab.append(current_string + current_format + '_' + label_text)
                count += 1
                if label_text.lower() == "mat":
                    match_records[current_format.lower()] = job_elem.text
                if current_string == "Batting_" and label_text.lower() == "runs":
                    try:
                        runs_scored_as_batsman += (int(job_elem.text))
                    except:
                        pass
                elif current_string == "Bowling_" and label_text.lower() == "balls":
                    try:
                        balls_bowled_as_bowler += (int(job_elem.text))
                    except:
                        pass
            else:
                current_format = job_elem.text
                if current_format[0] == 'W':
                    player_gender = "Female"
                    if current_format == "WTEST":
                        current_format = "Test"
                    else:
                        current_format = current_format[1:]
            track_head_label += 1
        match_averages.append(match_records)
    lab.append("Gender")
    result.append(player_gender)
    if(len(match_averages) != 2):
        return
    internationals = 0
    non_internationals = 0
    for key in match_averages[0]:
        if key in match_averages[1]:
            if key == "test" or key == "odi" or key == "t20i":
                try:
                    internationals += max(int(match_averages[0][key]), int(match_averages[1][key]))
                except:
                    pass
            else:
                try:
                    non_internationals += max(int(match_averages[0][key]), int(match_averages[1][key]))
                except:
                    pass
    if internationals == 0 and non_internationals < 15:
        return
    if runs_scored_as_batsman < 50 and balls_bowled_as_bowler < 50:
        return
    if ("bat" in player_role or "keep" in player_role) and runs_scored_as_batsman < 100:
        return
    if "bowl" in player_role and balls_bowled_as_bowler < 100:
        return
    if "round" in player_role and (runs_scored_as_batsman < 50 or balls_bowled_as_bowler < 50):
        return
    ####################################
    individual_player_id = str(req.url).split('-')[-1]
    lab.append("Cricinfo_id")
    result.append(individual_player_id)
    ####################################
    Relations = soup.find('p',class_="text-uppercase gray-700 mb-0 player-card-heading text-uppercase")
    #relation_players = soup.find('p',class_="player-card-description player-description-link gray-900 m-0 player-link")
    if(Relations != None):
        if(Relations.text.lower()=='relations' or Relations.text.lower()=='relation'):
            lab.append("Relations")
            all_relations = find_relations(Relations)
            result.append([all_relations])
    ######################################
    rec = extra(u)
    lab.append("Records")
    result.append([rec])
    if len(rec) > 0:
        all_player_urls.append(u)
    print(result)
    ##########################################
    records_format , records_data = format_records(u)
    for iter in range(len(records_format)):
      lab.append(records_format[iter])
      result.append([records_data[iter]])
    ###########################################
    matches = debut(debut_url)
    if len(matches) > 0:
        all_player_urls.append(debut_url)
    current_match_format = ""
    for match_detail in matches:
        match_detail = str(match_detail)
        if match_detail.startswith("Last"):
            lab.append(current_match_format + "_" + "last_appearance")
            result.append(match_detail[5:])
        elif match_detail.startswith("Debut"):
            lab.append(current_match_format + "_" + "debut")
            result.append(match_detail[6:])
        else:
            current_match_format = match_detail
            if current_match_format[0] == 'W':
                if current_match_format == "WTEST Matches":
                    current_match_format = "Test Matches"
                else:
                    current_match_format = current_match_format[1:]
    ##########################################
    lab.append("Teams")
    team_data = teams_data(soup)
    result.append([team_data])
    ###########################################
    info_labels, info_results = get_statistical_analysis_and_trophy_info_and_image(req)
    lab = lab + info_labels
    result = result + info_results
    if len(info_results) > 0:
        all_player_urls.append(req.url + "/bowling-batting-stats")
    ###########################################
    lab.append("References")
    result.append([all_player_urls])
    ####################################
    career_span_tag = soup.find('div', class_="intl_career-desktop")
    try:
      career_span_tag_text = career_span_tag.findNext('p').text
    except:
      career_span_tag_text = ""
    if len(career_span_tag_text) > 0:
        career_span_info = ""
        start_index = 0
        for i in range(len(career_span_tag_text)):
            if career_span_tag_text[i] == '1' or career_span_tag_text[i] == '2':
                start_index = i
                break
        career_span_tag_text = career_span_tag_text[start_index:]
        result.append(career_span_tag_text)
        lab.append("career_span")
    ###########################################
    search_tag_container = soup.find('div',class_="player-card__details")
    #player_name = search_tag_container.find("h2")
    country = search_tag_container.find("div",class_="player-card__country-name")
    #player_type = search_tag_container.find("div",class_="player-card__player-type")
    l_label = ["Nationality"]
    l = [country]
    result = result + l
    lab = lab + l_label
    ##########################
      
    print('label--',len(lab))
    print('result--',len(result))
    print(lab)
    print(result)
    lo = dict(zip(lab,result))
    print(lo)
    lp = pd.DataFrame(lo,index=None)
    print(lp)
    final_list_players.append(lp)
    print('count',count)

In [22]:
#Get individual player's "Records" information.
import re
def extra(l):
    re = requests.get(l)
    time.sleep(2)
    so = bf(re.text,"html.parser")
    res1 = so.find_all('div',class_='col-14 d-flex flex-row')
    res = so.find_all('div',class_='row bg-gray-100 class-record-row-p d-flex align-items-center')
    #print(res1)
    url_list = list()
    for i in range(len(res1)):

        pre = res1[i].find_all('h1')
        loi = res1[i].find_all('p',class_='record-class-title m-0')
        #print(loi)    
        for j in range(len(pre)):
            #print(pre[j].text)
            #print(loi[j].text)
            url_list.append(str(pre[j].text)  +str(" ") + str(loi[j].text))
    return url_list


In [23]:

def format_records(url):
  req = requests.get(url)
  soup = bf(req.text,"html.parser")
  record_sections = soup.find_all('div',class_="card content-block")
  required = ['Test Records','ODI Records','T20I Records']
  available_labels = list()
  labels_data = list()
  count = -1
  for i in range(len(record_sections)):
    name = record_sections[i].find('h5',class_="m-0")
    if(name.text in required):
      available_labels.append(name.text)
      labels_data.append([])
      count +=1
      individual_block_record = record_sections[i].find_all('a',class_="gray-900 benton-normal d-block")
      for i in range(len(individual_block_record)):
        try:
          a = individual_block_record[i].find('span',class_="m-0 benton-bold gray-1000 pr-3").text
          b = individual_block_record[i].find('span',class_="list-record-title").text
          c = individual_block_record[i].find('span',class_="m-0 ml-1 gray-600").text
          labels_data[count].append(str(a)+str(" ")+str(b)+str(" ")+str(c))
        except:
          no = 1
  return available_labels , labels_data

In [24]:
def teams_data(so):
  teams = []
  loi = so.find_all('h5',class_='m-0 ml-2 link-border-bottom player-description-link')
  for i in range(len(loi)):
    teams.append(loi[i].text)
  return teams

In [25]:
def debut(link):
  player_match = list()
  url = link
  request = requests.get(url)
  soup = bf(request.text,"html.parser")
  search_tag_container = soup.find_all('div',{'id':'debut-last-matches'})
  cp_jo = search_tag_container
  if(len(search_tag_container) != 0):
    total = cp_jo[0].find_all('div',class_="")
    for i in range(len(total)):
      p =total[i]
      a= p.find('h5',{'class':"benton-bold player-matches-subtitle d-flex justify-content-between align-items-center player_matches-dropdown m-0"})
      k = total[i].find_all('span',class_="match-event-row-debut-last white-space-nowrap")
      search_tag_container = total[i].find_all('span',{"class": "player-match-link"})
      new = total[i].find_all('div',{"class":"col-12 player-match-event-cell white-space-nowrap"})
      #print(a)
      #print(k[0])
      #print(new)
      try:
        l =a.span.decompose()
      except:
        player_match.append(a.text)
        #print(a.text)
      else:
        player_match.append(a.text)
        #print(a.text)
      for j in range(len(k)):
        try:
          temp = search_tag_container[j].text
        except:
          temp = new[j].text
        append = k[j].text + str(" ") + temp
        player_match.append(append)
        #print(k[j].text,end=" ")
        #print(search_tag_container[j].text)
  return player_match

In [26]:

import time
final_list_players = list()
for country_id in range(16,28):
  #Country based parse url
  for letter in range(65,91):
    url = "https://www.espncricinfo.com/ci/content/player/country.html?country="+str(country_id)+";alpha="+str(chr(letter))
    req = requests.get(url)
    print(url)
    soup = bf(req.text,"html.parser")
    #Fetch total urls of players belonging to a country
    #country_name = soup.find('span',{'id':'ciShowCountry'})
    country_name = soup.find('div',{"class":"icc-home"})

    print("-------------------------------------------------")
    print(country_name.text)
    print("-------------------------------------------------")
    search_tag_container = soup.find_all('td')
    u = list()
    for job_elem in search_tag_container:
        u.append(job_elem.find('a'))
        #result.append(job_elem.text)
    l = set(u)
    l = list(l)
    for i in range(len(l)):
        u = "https://www.espncricinfo.com"
        rl = l[i]["href"]
        link = u+rl
        print(link)
        #link = "https://www.espncricinfo.com/player/virat-kohli-253802"
        req = requests.get(link)
        time.sleep(2)
        print('player-',i,end=" ")
        ind = len(final_list_players)
        record_url = req.url+str("/tests-odi-t20-records")
        debut_url = req.url+str("/matches")
        #debut(debut_url)
        load(req,final_list_players,ind,record_url,debut_url)
        #print(record_url)
        #jk = extra(record_url)
        #print(jk)
    try:
      driver.close()
    except:
      no_drive = 1

https://www.espncricinfo.com/ci/content/player/country.html?country=16;alpha=A
-------------------------------------------------


Players / Malaysia / A - Z
				

-------------------------------------------------
https://www.espncricinfo.com/ci/content/player/1037595.html
player- 0 <title>Arjuna profile and biography, stats, records, averages, photos and videos</title>
https://www.espncricinfo.com/ci/content/player/307647.html
player- 1 <title>Sarath Ananthasivam profile and biography, stats, records, averages, photos and videos</title>
https://www.espncricinfo.com/ci/content/player/1174345.html
player- 2 <title>Amalin Sorfina profile and biography, stats, records, averages, photos and videos</title>
Head count 15
Head count 14
https://www.espncricinfo.com/ci/content/player/1050039.html
player- 3 <title>Aina Najwa profile and biography, stats, records, averages, photos and videos</title>
Head count 15
Head count 14
https://www.espncricinfo.com/ci/content/player/1021053.html
player- 4 

TypeError: ignored

In [ ]:
a = final_list_players[0]
df = a.T
for i in range(1,len(final_list_players)):
    lo = final_list_players[i].T
    df = pd.concat([df,lo],axis=1)
final_df = df.T
final_df.to_csv("output_players.csv", index=False)